# X-ray Imaging and Reconstruction

* The following examples simulate recording x-ray projections and their reconstruction by applying the inverse radon transform in a simplified manner

## Simple Geometric Objects

* The following snippet allows to place different simple geometric objects into the image plane, i.e. uniform squares or discs of different size, number and random position.
* Widgets like sliders and buttons can be used to visualize the impact on the reconstruction
  * Number of projections
  * Type of reconstruction filter to compare backprojection (None) against filtered backprojection (any other filter)
* Some parameters are fixed
  * Angle range is from 0° to 180°
  * Image plane is 256x256 pixels

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from skimage.draw import disk, rectangle
from skimage.transform import radon, iradon
from ipywidgets import interact, IntSlider, Dropdown, ToggleButton, interactive_output, VBox,


In [19]:

# Global variable to store random seed for reproducibility between redistributions
current_random_seed = np.random.randint(0, 1e6)

# Function to generate a scene with squares or discs
def generate_scene(object_type, num_objects, object_size, seed=None):
    if seed is not None:
        np.random.seed(seed)
    scene = np.zeros((256, 256), dtype=np.float32)
    
    # Randomly place objects
    for _ in range(num_objects):
        center_x, center_y = np.random.randint(0, 256, 2)
        
        if object_type == "Squares":
            # Define square bounds and draw it
            half_size = object_size // 2
            top_left = (max(center_x - half_size, 0), max(center_y - half_size, 0))
            bottom_right = (min(center_x + half_size, 255), min(center_y + half_size, 255))
            rr, cc = rectangle(start=top_left, end=bottom_right)
            scene[rr, cc] = 1  # Uniform square intensity
        
        elif object_type == "Discs":
            # Define and draw a disc
            rr, cc = disk((center_x, center_y), radius=object_size // 2, shape=scene.shape)
            scene[rr, cc] = 1  # Uniform disc intensity
    
    return scene

# Function to simulate the Radon transform, projections, and reconstruction
def simulate_reconstruction(object_type, num_objects, object_size, num_angles, filter_type, selected_projection, redistribute_objects):
    global current_random_seed
    # If redistribution is requested, update the seed to get a new random configuration
    if redistribute_objects:
        current_random_seed = np.random.randint(0, 1e6)
        redistribute_toggle_button.value = False  # Reset toggle button after redistribution
    
    # Generate the scene with squares or discs using the current random seed
    scene = generate_scene(object_type, num_objects, object_size, seed=current_random_seed)
    
    # Generate projection angles
    theta = np.linspace(0., 180., num_angles, endpoint=False)
    
    # Radon transform (simulating projections)
    sinogram = radon(scene, theta=theta, circle=True)
    
    # Reconstruction using Filtered Back Projection (FBP)
    reconstructed_image = iradon(sinogram, theta=theta, filter_name=filter_type, circle=True)
    
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    ax = axes.ravel()
    
    # Plot the original scene
    ax[0].set_title("Original Scene")
    ax[0].imshow(scene, cmap='gray')
    ax[0].axis('off')
    
    # Plot the sinogram with a vertical line indicating the selected projection
    ax[1].set_title(f"Sinogram (Angles: {num_angles})")
    ax[1].imshow(sinogram, cmap='gray', aspect='auto')
    ax[1].axvline(x=selected_projection, color='red', linestyle='--')
    ax[1].set_xlabel("Projection angle (index)")
    ax[1].set_ylabel("Projection position (pixels)")
    
    # Plot the reconstructed image
    ax[2].set_title(f"Reconstruction\n(Filter: {filter_type})")
    ax[2].imshow(reconstructed_image, cmap='gray')
    ax[2].axis('off')
    
    # Plot the selected projection line
    projection_data = sinogram[:, selected_projection]
    ax[3].set_title(f"Projection at Angle {theta[selected_projection]:.1f}°")
    ax[3].plot(projection_data, color='blue')
    ax[3].set_xlabel("Position (pixels)")
    ax[3].set_ylabel("Intensity")
    
    plt.tight_layout()
    plt.show()

# Interactive widgets with dynamic slider for selected_projection

object_type_dropdown = Dropdown(
    options=["Squares", "Discs"],
    value="Squares",
    description="Object Type"
)
num_objects_slider = IntSlider(value=3, min=1, max=10, step=1, description="Number of Objects")
object_size_slider = IntSlider(value=20, min=0, max=50, step=1, description="Object Size")
num_angles_slider = IntSlider(value=30, min=1, max=256, step=1, description="Number of Angles")
filter_type_dropdown = Dropdown(
    options=["ramp", "shepp-logan", "cosine", "hamming", "hann", None],
    value="ramp",
    description="Filter"
)


# Initialize selected_projection slider
selected_projection_slider = IntSlider(value=0, min=0, max=29, step=1, description="Projection Index")

# Function to update the max value of selected_projection slider based on num_angles
def update_selected_projection_slider(num_angles):
    selected_projection_slider.max = max(0, num_angles - 1)

# Link num_angles to selected_projection max update
num_angles_slider.observe(lambda change: update_selected_projection_slider(change['new']), names='value')

# Toggle button to trigger random redistribution of objects
redistribute_toggle_button = ToggleButton(value=False, description="Redistribute Objects")

# Set up the interactivity with dynamic dependency
interactive_plot = interactive_output(
    simulate_reconstruction,
    {
        'object_type': object_type_dropdown,
        'num_objects': num_objects_slider,
        'object_size': object_size_slider,
        'num_angles': num_angles_slider,
        'filter_type': filter_type_dropdown,
        'selected_projection': selected_projection_slider,
        'redistribute_objects': redistribute_toggle_button
    }
)


* Move index of selected projection to 0 (or below number of projections) when decreasing number of projections

In [20]:

# Display all widgets and output
display(VBox([
    object_type_dropdown, 
    num_objects_slider, 
    object_size_slider, 
    num_angles_slider, 
    filter_type_dropdown, 
    selected_projection_slider, 
    redistribute_toggle_button,
    interactive_plot
]))


## Shepp-Logan Phantom

* Same with so called Shepp-Logan phantom (numerically ressembling slice of head)

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.data import shepp_logan_phantom
from skimage.transform import radon, iradon
from ipywidgets import interact, IntSlider, Dropdown, interactive_output


In [21]:

# Generate the Shepp-Logan phantom image
phantom = shepp_logan_phantom()
phantom = phantom[::2, ::2]  # Resize to make it faster for demonstration

# Function to perform Radon transform, simulate projections, and reconstruction
def simulate_reconstruction(num_angles, filter_type, selected_projection):
    # Generate projection angles
    theta = np.linspace(0., 180., num_angles, endpoint=False)
    
    # Radon transform (simulating projections)
    sinogram = radon(phantom, theta=theta, circle=True)
    
    # Reconstruction using Filtered Back Projection (FBP)
    reconstructed_image = iradon(sinogram, theta=theta, filter_name=filter_type, circle=True)
    
    # Plot the results
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    ax = axes.ravel()
    
    # Plot the original phantom
    ax[0].set_title("Original Phantom")
    ax[0].imshow(phantom, cmap='gray')
    ax[0].axis('off')
    
    # Plot the sinogram with a vertical line indicating the selected projection
    ax[1].set_title(f"Sinogram (Angles: {num_angles})")
    ax[1].imshow(sinogram, cmap='gray', aspect='auto')
    ax[1].axvline(x=selected_projection, color='red', linestyle='--')
    ax[1].set_xlabel("Projection angle (index)")
    ax[1].set_ylabel("Projection position (pixels)")
    
    # Plot the reconstructed image
    ax[2].set_title(f"Reconstruction\n(Filter: {filter_type})")
    ax[2].imshow(reconstructed_image, cmap='gray')
    ax[2].axis('off')
    
    # Plot the selected projection line
    projection_data = sinogram[:, selected_projection]
    ax[3].set_title(f"Projection at Angle {theta[selected_projection]:.1f}°")
    ax[3].plot(projection_data, color='blue')
    ax[3].set_xlabel("Position (pixels)")
    ax[3].set_ylabel("Intensity")
    
    plt.tight_layout()
    plt.show()

# Interactive widgets with dynamic slider for selected_projection
num_angles_slider = IntSlider(value=30, min=1, max=256, step=1, description="Number of Angles")
filter_type_dropdown = Dropdown(
    options=["ramp", "shepp-logan", "cosine", "hamming", "hann", None],
    value="ramp",
    description="Filter"
)

# Function to update the max value of selected_projection slider
def update_selected_projection_slider(num_angles):
    selected_projection_slider.max = max(0, num_angles - 1)

# Initialize selected_projection slider
selected_projection_slider = IntSlider(value=0, min=0, max=29, step=1, description="Projection Index")

# Set up the interactivity with dynamic dependency
interactive_plot = interactive_output(
    simulate_reconstruction,
    {'num_angles': num_angles_slider,
     'filter_type': filter_type_dropdown,
     'selected_projection': selected_projection_slider}
)

# Link num_angles to selected_projection max update
num_angles_slider.observe(lambda change: update_selected_projection_slider(change['new']), names='value')


* Move index of selected projection to 0 (or below number of projections) when decreasing number of projections

In [22]:

# Display all widgets and output
display(num_angles_slider, filter_type_dropdown, selected_projection_slider, interactive_plot)


IntSlider(value=30, description='Number of Angles', max=256, min=1)

Dropdown(description='Filter', options=('ramp', 'shepp-logan', 'cosine', 'hamming', 'hann', None), value='ramp…

IntSlider(value=0, description='Projection Index', max=29)

Output()